In [1]:
from custom_agent.CTCE.sac_agent_AE import Agent
from custom_agent.CTCE.citylearn_wrapper import CityLearnWrapper

from citylearn.wrappers import NormalizedSpaceWrapper
from custom_reward.custom_reward import CustomReward

from citylearn.citylearn import CityLearnEnv

In [2]:
class WrapperEnv:
    """
    Env to wrap provide Citylearn Env data without providing full env
    Preventing attribute access outside of the available functions
    """
    def __init__(self, env_data):
        self.observation_names = env_data['observation_names']
        self.action_names = env_data['action_names']
        self.observation_space = env_data['observation_space']
        self.action_space = env_data['action_space']
        self.time_steps = env_data['time_steps']
        self.seconds_per_time_step = env_data['seconds_per_time_step']
        self.random_seed = env_data['random_seed']
        self.buildings_metadata = env_data['buildings_metadata']
        self.episode_tracker = env_data['episode_tracker']
    
    def get_metadata(self):
        return {'buildings': self.buildings_metadata}
    
def makeEnv(schema_path, reward_function):
    # create environment
    env = CityLearnEnv(schema = schema_path, reward_function = reward_function, central_agent=True)

    env_data = dict(
        observation_names = env.observation_names,
        action_names = env.action_names,
        observation_space = env.observation_space,
        action_space = env.action_space,
        time_steps = env.time_steps,
        random_seed = None,
        episode_tracker = None,
        seconds_per_time_step = None,
        buildings_metadata = env.get_metadata()['buildings']
    )

    wrapper_env = WrapperEnv(env_data)
    return env, wrapper_env

In [3]:
schema_path = "data/schema.json"

env, wrapper_env = makeEnv(schema_path, CustomReward)
env = NormalizedSpaceWrapper(env)
env = CityLearnWrapper(env)

In [4]:

logs_list = []
learn_delay = 100

for i in range(1):
    # make agent
    sac_agent = Agent(env, batch_size=256, buffer_max_size=100000)
    # sac_agent = SAC("MlpPolicy", env)
    
    # training run
    sac_agent.train(3950000, warmup_steps=100, learn_delay = 100, learn_freq = 1, learn_weight = 1)

/home/hkolstee/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/hkolstee/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hkolstee/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


[Episode 50 total reward: -475.718] ~ 
[Episode 100 total reward: -529.945] ~ 
Load factor
Load factor
Load factor
[Episode 150 total reward: -727.769] ~ 
[Episode 200 total reward: -777.208] ~ 
[Episode 250 total reward: -877.141] ~ 
[Episode 300 total reward: -750.284] ~ 
[Episode 350 total reward: -830.385] ~ 
[Episode 400 total reward: -692.962] ~ 
[Episode 450 total reward: -723.037] ~ 
[Episode 500 total reward: -710.349] ~ 
[Episode 550 total reward: -643.316] ~ 
[Episode 600 total reward: -608.605] ~ 
[Episode 650 total reward: -595.211] ~ 
[Episode 700 total reward: -687.918] ~ 
[Episode 750 total reward: -719.200] ~ 
[Episode 800 total reward: -720.404] ~ 
[Episode 850 total reward: -506.623] ~ 
[Episode 900 total reward: -602.766] ~ 
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/hkolstee/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_21926/2374753819.py", line 10, in <module>
    sac_agent.train(3950000, warmup_steps=100, learn_delay = 100, learn_freq = 1, learn_weight = 1)
  File "/home/hkolstee/masterThesis/masterThesis/custom_agent/CTCE/sac_agent_AE.py", line 294, in train
  File "/home/hkolstee/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/hkolstee/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/hkolstee/masterThesis/masterThesis/custom_agent/SAC_components/autoencoder.py", line 45, in forward
    # this forward function automatically also takes a step down the gradient (convenient)


In [ ]:
from gymnasium.utils.save_video import save_video

# test loop
obs, info = env2.reset()
done = 0
truncated = 0
while not (done or truncated):
    action = sac_agent.get_action(obs, False, True)

    next_obs, reward, done, truncated, info = env2.step(action)

    obs = next_obs

save_video(env2.render(), "videos", name_prefix = "custom_SAC_halfcheetahv4", fps=env.metadata["render_fps"], step_starting_index=0)